In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import joblib

X=np.load('xlo.npy')[:,[9,6,34,8,3,20,15,14,5,1,21,12,26,4,11,13,10,7,2,23,0]] 
y=np.load('ylo1.npy')[:]
X_test1=np.load('xlo_test.npy')[:,[9,6,34,8,3,20,15,14,5,1,21,12,26,4,11,13,10,7,2,23,0]] 
y_test1=np.load('ylo_test1.npy')[:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


X_train_selected = X_train[:, :]
X_test_selected = X_test[:, :]
svm = SVC(kernel='linear', C=100, class_weight='balanced', random_state=42)
svm.fit(X_train_selected, y_train)
joblib.dump(svm, 'svm_model.joblib')
accuracy = svm.score(X_test_selected, y_test)
y_pred = svm.predict(X_test_selected)
f1 = f1_score(y_test, y_pred, average=None)
y_pred = svm.predict(X_test_selected)
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
print(f"Class 0 F1 Score: {f1[0]:.4f}")
print(f"Class 1 F1 Score: {f1[1]:.4f}")
print(f"Accuracy: {accuracy:.2f}") 
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Negatives (TN): {tn}")


Ax=np.zeros([0,X.shape[1]])
Ay=[] 
y_pred_train = svm.predict(X_train_selected)
for i in range(y_pred_train.shape[0]):
    if np.remainder(i,10000)==0:
        print('i=',i)
    if y_train[i]==1:
        Ax=np.append(Ax,X_train_selected[[i],:],axis=0)
        Ay=np.append(Ay,y_train[i])
    if y_train[i]==0:
        if y_pred_train[i]==1:
            Ax=np.append(Ax,X_train_selected[[i],:],axis=0)
            Ay=np.append(Ay,y_train[i])

            
svm1 = SVC(kernel='linear', C=100, class_weight='balanced', random_state=42)
svm1.fit(Ax, Ay)
joblib.dump(svm1, 'svm1_model.joblib')
y_pred_train = svm1.predict(Ax)
f1 = f1_score(Ay, y_pred_train, average=None)
cm = confusion_matrix(Ay, y_pred_train, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()     
y_pred_test = svm1.predict(X_test)
f11 = f1_score(y_test, y_pred_test, average=None)
cm1 = confusion_matrix(y_test, y_pred_test, labels=[0, 1])
tn1, fp1, fn1, tp1 = cm1.ravel() 
y_pred_test1 = svm1.predict(X_test1)
f12 = f1_score(y_test1, y_pred_test1, average=None)
cm2 = confusion_matrix(y_test1, y_pred_test1, labels=[0, 1])
tn2, fp2, fn2, tp2 = cm2.ravel()
output=[[f1[0],f1[1],tn, fp, fn, tp],[f11[0],f11[1],tn1, fp1, fn1, tp1],[f12[0],f12[1],tn2, fp2, fn2, tp2]]
with open('output.txt','wb') as f:
    np.savetxt(f,output)